In [10]:
pip install xlrd==1.2.0

In [11]:
pip install xlsxwriter

In [12]:
import pandas as pd
import numpy as np
import xlrd
from sklearn.preprocessing import  MinMaxScaler 
from sklearn.neural_network import  MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix , precision_score, recall_score 
from sklearn.metrics import jaccard_score, f1_score
#### import data
workbook = xlrd.open_workbook("../input/file-11/training set.xlsx","rb")
sheets = workbook.sheet_names()
sheet_num=1  ## add sheet number # 5
sh = workbook.sheet_by_name(sheets[(sheet_num)-1])
n_cols=0 ##number of colum you have to delet from the front
n_rows=0 ##number of rows you have to delet from the front 
m_cols=0 ## number of colum you have to delet from the end
m_rows=0 ## number of rows you have to delet from the end
M_rows=sh.nrows-m_rows
M_cols=sh.ncols-m_cols
required_data = []
for rownum in range(n_rows,M_rows):
    row_valaues = sh.row_values(rownum)
    required_data.append(row_valaues[n_cols:M_cols])
Required_data=np.asarray(required_data)
X=Required_data[1:82334,1:40].astype(float)  #  25194 # 82334  ( 4:41 & 1:40)
X= np.array (X)

Y=Required_data[1:82334,41:42].astype(float).ravel()    # (42:43  & 41:42)
print(X.shape)

bands = Required_data[0:1, 1:40]      # ( 4:41 & 1:40)
bands= np.array (bands)

X = MinMaxScaler().fit(X).transform(X)
#===============
X_train, X_test, Y_train, Y_test = split(X, Y, test_size=(0.20), random_state=0)

layer1_f = []
layer2_f = []
activation_f = []
r2_train_f =[]
r2_test_f = []
coefficient_features =[]
features_best = []
loss_train_f=[]
loss_test_f=[]

index = min(X_train.shape) 

for j in range (0,index): # 
    n=min(X_train.shape) 
    layer11 = []
    layer22 = []
    activation1 = []
    r2_train1 =[]
    r2_test1 = []
    loss_train1 = []
    loss_test1 = []    
    layer1 = 8
    layer2 = 10
    for activation in  ('identity','logistic', 'tanh', 'relu'): 
        
        print (activation) 

        model = MLPClassifier(hidden_layer_sizes=(layer1,layer2),
                                  activation=activation,
                                  solver='lbfgs',
                                  max_iter=300,
                                  verbose=False, 
                                  alpha=1e-5, 
                                  tol=1e-5, 
                                  random_state=42)  
            
        BPNN_model=model.fit(X_train, Y_train)
        #==========
        Y_train_pred=BPNN_model.predict(X_train)  
        Y_test_pred=BPNN_model.predict(X_test)
        #===========
        loss_train = log_loss(Y_train, Y_train_pred)
        loss_test = log_loss(Y_test, Y_test_pred)
        #===========
        r2_train= accuracy_score(Y_train, Y_train_pred)
        r2_test= accuracy_score(Y_test, Y_test_pred)
        #==========
        print ("r2_train",r2_train, 'loss_train', loss_train)
        print ("r2_test",r2_test, 'loss_test', loss_test)                
        layer11.append (layer1)
        layer22.append (layer2) 
        activation1.append (activation)
        r2_train1.append (r2_train)
        r2_test1.append (r2_test)
        loss_train1.append (loss_train)
        loss_test1.append (loss_test)                
        
        print ('********************************')     
    
    loss_test_min = np.argmin (loss_test1)
    layer1 = layer11[loss_test_min]
    layer2 = layer22 [loss_test_min]
    activation = activation1 [loss_test_min]                                
    r2_train= r2_train1 [loss_test_min] 
    r2_test = r2_test1 [loss_test_min] 
    loss_train = loss_train1 [loss_test_min] 
    loss_test = loss_test1 [loss_test_min] 
    
    layer1_f.append (layer1)
    layer2_f.append (layer2)
    activation_f.append (activation)
    r2_train_f.append (r2_train)
    r2_test_f.append (r2_test)
    loss_train_f.append (loss_train)
    loss_test_f.append (loss_test)
    
    model = MLPClassifier(hidden_layer_sizes=(layer1,layer2),
                              activation=activation,
                              solver='lbfgs',
                              max_iter=300,
                              verbose=False, 
                              alpha=1e-5, 
                              tol=1e-5, 
                              random_state=42)    
    
    BPNN_model=model.fit(X_train, Y_train)
    
             
    #~~~~~~  select impotance features according to Eq. Glorfeld, 1996
    W_1= MinMaxScaler().fit(abs(BPNN_model.coefs_[0]).T).transform(abs(BPNN_model.coefs_[0]).T) ## the weight between input layers and first hidden layer after normalization
        
    numerator=np.sum((abs(BPNN_model.coefs_[0])/np.sum(abs(BPNN_model.coefs_[0]), axis=0))*(np.sum(abs(BPNN_model.coefs_[1]), axis=1)), axis=1)

    M=numerator/sum(numerator)    ## important features
    Mm=MinMaxScaler().fit(M.reshape(-1, 1)).transform(M.reshape(-1, 1))

    sorted_ind = np.argsort(np.abs(Mm[:,0])) # الاصغر فالاكبر
    X_train = X_train[:,sorted_ind]
    bands = bands [:,sorted_ind]
    coefficient_features.append(Mm) 
    #~~~~~~~~~~~~~~~~~~
    print ("number of indices = ", min (X_train.shape) )
    print ('The end-------------------------------------------')
    features_best.append ([bands])
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    print ('The second loop')   
    X_train = X_train[:, 1:]
    X_test = X_test[:, 1:]
    bands = bands [:, 1:]
        
#~~~ the higher R2_test
loss_test_min_f = np.argmin (loss_test_f) 
#~~~~~~ best parameter
layer1_ = layer1_f [loss_test_min_f]
layer2_ = layer2_f [loss_test_min_f]
activation_ = activation_f [loss_test_min_f]
r2_train_f_ =  r2_train_f [loss_test_min_f]
r2_test_f_ = r2_test_f [loss_test_min_f]
loss_test_f_ = loss_train_f [loss_test_min_f]
loss_train_f_ = loss_test_f [loss_test_min_f]
           
features_best_ = features_best [loss_test_min_f]
coefficient_features_ = coefficient_features [loss_test_min_f]

print ("layer1 =",layer1_)
print ("layer2 =",layer2_)
print ('activat=', activation_)
print ('r2_test=', r2_test_f_)
print ('r2_train=', r2_train_f_)
print ('loss_test=', loss_test_f_)
print ('loss_train=', loss_train_f_)
print ("The best band =",features_best_)
print ('====================')
#~~~~~~~
layer1 = pd.DataFrame(layer1_f)
layer2 = pd.DataFrame(layer2_f)
activation = pd.DataFrame(activation_f)
r2_train = pd.DataFrame (r2_train_f)
r2_test = pd.DataFrame (r2_test_f)
features = pd.DataFrame (features_best)
coefficient = pd.DataFrame (coefficient_features)
loss_train = pd.DataFrame (loss_train_f)
loss_test = pd.DataFrame (loss_test_f)

writer = pd.ExcelWriter("BPNN_feature selection_data2" + ".xlsx", engine='xlsxwriter')
layer1.to_excel(writer, sheet_name='layer1', index=False, header=0)
layer2.to_excel(writer, sheet_name='layer2', index=False, header=0)
activation.to_excel(writer, sheet_name='activation', index=False, header=0)
r2_train.to_excel(writer, sheet_name='r2_train', index=False, header=0)
r2_test.to_excel(writer, sheet_name='r2_test', index=False, header=0)
loss_train.to_excel(writer, sheet_name='loss_train', index=False, header=0)
loss_test.to_excel(writer, sheet_name='loss_test', index=False, header=0)
features.to_excel(writer, sheet_name='features', index=False, header=0)
coefficient.to_excel(writer, sheet_name='coefficient', index=False, header=0)
writer.save() 

In [15]:
import pandas as pd
import numpy as np
import xlrd
import matplotlib.pyplot as plt
from sklearn.preprocessing import  MinMaxScaler 
from sklearn.neural_network import  MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix , precision_score, recall_score 
from sklearn.metrics import jaccard_score, f1_score
import time
import pickle

time1=time.time()

#### import data
workbook = xlrd.open_workbook("../input/file-11/training set.xlsx","rb")
sheets = workbook.sheet_names()
sheet_num=1  ## add sheet number # 5
sh = workbook.sheet_by_name(sheets[(sheet_num)-1])
n_cols=0 ##number of colum you have to delet from the front
n_rows=0 ##number of rows you have to delet from the front 
m_cols=0 ## number of colum you have to delet from the end
m_rows=0 ## number of rows you have to delet from the end
M_rows=sh.nrows-m_rows
M_cols=sh.ncols-m_cols
required_data = []
for rownum in range(n_rows,M_rows):
    row_valaues = sh.row_values(rownum)
    required_data.append(row_valaues[n_cols:M_cols])
Required_data=np.asarray(required_data)
X=Required_data[1:82334,1:40].astype(float)  #  25194 # 82334  ( 4:41 & 1:40)
X= np.array (X)

Y=Required_data[1:82334,41:42].astype(float).ravel()    # (42:43  & 41:42)
print(X.shape)

bands = Required_data[0:1, 1:40]      # ( 4:41 & 1:40)
bands= np.array (bands)

X = MinMaxScaler().fit(X).transform(X)
#===============
X_train, X_test, Y_train, Y_test = split(X, Y, test_size=(0.20), random_state=0)

layer1 = 8     # please put the best selected number 
layer2 =  10       # please put the best selected number 
activation = 'tanh'       # please put the best selected function 


model = MLPClassifier(hidden_layer_sizes=(layer1,layer2),
                          activation=activation,
                          solver='lbfgs',
                          max_iter=300,
                          verbose=False, 
                          alpha=1e-5, 
                          tol=1e-5, 
                          random_state=42)    

BPNN_model=model.fit(X_train, Y_train)

print ((time.time()-time1))

# save the model to disk
filename = 'BPNN_data_2_model.h5'
pickle.dump(BPNN_model, open(filename, 'wb'))
 


Y_train_pred=BPNN_model.predict(X_train)  
Y_test_pred=BPNN_model.predict(X_test)
#===========
loss_train = log_loss(Y_train, Y_train_pred)
loss_test = log_loss(Y_test, Y_test_pred)
#===========
r2_train= accuracy_score(Y_train, Y_train_pred)
r2_test= accuracy_score(Y_test, Y_test_pred)
#==========
print ("r2_train",r2_train, 'loss_train', loss_train)
print ("r2_test",r2_test, 'loss_test', loss_test)   

CM = confusion_matrix(Y_test, Y_test_pred)
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
#================ classification metrics
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
print ('TPR',TPR)   
# Specificity or true negative rate
TNR = TN/(TN+FP) 
print('TNR',TNR)
# Precision or positive predictive value
PPV = TP/(TP+FP)
print ('PPV', PPV)
# Negative predictive value
NPV = TN/(TN+FN)
print('NPV', NPV)
# Fall out or false positive rate
FPR = FP/(FP+TN)
print('FPR',FPR)
# False negative rate
FNR = FN/(TP+FN)
print ('FNR',FNR)  
# False discovery rate
FDR = FP/(TP+FP)
print ('FDR',FDR)    
# Overall accuracy   
ACC = (TP+TN)/(TP+FP+FN+TN)
print ('ACC',ACC)

precision = precision_score(Y_test, Y_test_pred, average='weighted')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_test, Y_test_pred, average='weighted')
print('Recall: %f' % recall)
# f1: tp / (tp + fp + fn)
f1 = f1_score(Y_test, Y_test_pred, average='weighted')
print('F1 score: %f' % f1)
#-----------  IoU
print ('IoU:', jaccard_score(Y_test, Y_test_pred, average='micro'))

In [17]:

X_train, X_test, Y_train, Y_test=split(X, Y, test_size=(0.20), random_state=0)

RMSE_test = []
r2_val11 = []
r2_train =[]
r2_test = []
Difference = []
mse_cv = []
Y_test_val_cv = []
Y_test_val_pred_cv = []
rpd_cv1 = []
RMSEC_train = []
r2_val =[]
#~~~~~~~~~~~~~
Yprdict_test = []
Ytrue_test = []
#~~~~~~~~~~~~~
N_EPOCHS = 301
epoch = 1
#~~~~~~~~~~~~
layer1 = 8    
layer2= 12
activa = 'tanh'   # identity','logistic', 'tanh', 'relu'

r2_train1 =[]
r2_test1 = []
loss_train1 = []
loss_test1 = [] 
    
while epoch < N_EPOCHS:
    print('epoch: ', epoch)
    
    model = MLPClassifier(hidden_layer_sizes=(layer1,layer2),
                              activation=activation,
                              solver='lbfgs',
                              max_iter=epoch,
                              verbose=False, 
                              alpha=1e-5, 
                              tol=1e-5, 
                              random_state=42)    
    
    BPNN_model=model.fit(X_train, Y_train)
    
    Y_train_pred=BPNN_model.predict(X_train)  
    Y_test_pred=BPNN_model.predict(X_test)
    #==========
    print ('epoch', epoch)
    print ("r2_train",r2_train, 'loss_train', loss_train)
    print ("r2_test",r2_test, 'loss_test', loss_test) 
    #===========
    loss_train = log_loss(Y_train, Y_train_pred)
    loss_test = log_loss(Y_test, Y_test_pred)
    loss_train1.append (loss_train)
    loss_test1.append (loss_test)
    #===========
    r2_train= accuracy_score(Y_train, Y_train_pred)
    r2_test= accuracy_score(Y_test, Y_test_pred)
    r2_train1.append (r2_train)
    r2_test1.append (r2_test)
    epoch += 1
    

import pandas as pd
r2_train1= pd.DataFrame(data= r2_train1) 
r2_test1= pd.DataFrame(data= r2_test1) 

loss_train1= pd.DataFrame(data= loss_train1) 
loss_test1 = pd.DataFrame(data= loss_test1)

""" save file to plot the results manually """

writer = pd.ExcelWriter("Epoch_data_2"+ ".xlsx", engine='xlsxwriter')
r2_train1.to_excel(writer, sheet_name='r2_train1', index=False, header=0)# all R2 in the sheet data
r2_test1.to_excel(writer, sheet_name='r2_test1', index=False, header=0)# all R2 in the sheet data
loss_train1.to_excel(writer, sheet_name='loss_train1', index=False, header=0)# all R2 in the sheet data
loss_test1.to_excel(writer, sheet_name='loss_test1', index=False, header=0)# all R2 in the sheet data
writer.save()

""" Plot """

epochs=range(1, len(r2_train1)+1)

font = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'normal',
        'size'   : 12}

loss_train1 = np.array (loss_train1)
loss_test1 = np.array (loss_test1)

plt.plot(epochs, loss_train1, 'r', label='Training')
plt.plot(epochs, loss_test1, 'g', label='Validation')
plt.title('Training and Validation Loss', fontdict=font)
plt.xlabel('Epochs', fontdict=font)
plt.ylabel('Loss', fontdict=font)
plt.legend()
plt.savefig("loss.png")
plt.show()
#==========

r2_train1 = np.array (r2_train1)
r2_test1 = np.array (r2_test1)

plt.plot(epochs, r2_train1, 'r', label='Training')
plt.plot(epochs, r2_test1, 'g', label='Validation')
plt.title('Training and Validation Score', fontdict=font)
plt.xlabel('Epochs', fontdict=font)
plt.ylabel('Accuracy', fontdict=font)
plt.legend()
plt.savefig("accuracy.png")
plt.show()


In [21]:
class_names = ['normal','Attack']


def plot_confusion_matrix(y_true, y_pred, classes,normalize=False,title=None, cmap=plt.cm.Reds):
                                              
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = None    ### 'Normalized confusion matrix'
        else:
            title = None        ### 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(Y_test, Y_test_pred)
    # Only use the labels that appear in the data
#    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize=(8,8))
    ax.tick_params(labelsize=12)       #777777777777
    cmap=plt.cm.Reds                      #  cmap=plt.cm.Blues
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    cbar = ax.figure.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    for t in cbar.ax.get_yticklabels():
        t.set_fontsize(12)
    
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
             
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

font = {'family' : 'Times New Roman',
        'color'  : 'black',
        'weight' : 'bold',
        'size'   : 18}

np.set_printoptions(precision=2) ########################## 2

# Plot non-normalized confusion matrix
plot_confusion_matrix(Y_test, Y_test_pred, classes=class_names,title= None)             
plt.xlabel('Predicted label', fontdict=font)
plt.ylabel('True label',fontdict=font)
plt.savefig('confusion matrix_1_.png')
plt.show()

# Plot normalized confusion matrix
plot_confusion_matrix(Y_test, Y_test_pred, classes=class_names, normalize=True,title=None)            
plt.xlabel('Predicted label', fontdict=font)
plt.ylabel('True label',  fontdict=font)
plt.savefig('confusion matrix_2_.png')
plt.show()
